In [155]:
import gensim
import pandas as pd
from gensim import corpora
from gensim import models
import pyLDAvis.gensim
from gensim.models import LdaModel
from gensim.corpora.dictionary import Dictionary
import matplotlib.pyplot as plt
import numpy as np
from gensim.models import CoherenceModel
import warnings
warnings.filterwarnings("ignore")

In [42]:
df = pd.read_csv('database_sentiment_label.csv',parse_dates=['publishedAt','extractedAt'])
df.head()

,channelId,videoId,author,publishedAt,comment,extractedAt,tokens_clean_lem,comment_clean_lem,sentiment_label
0,UCK3inMNRNAVUleEbpDU1k2g,24Ob5JxhF4s,@SEBFRIT,2025-06-02 19:21:45+00:00,Merci à Odoo le sponsor cette vidéo ! Pour déc...,2025-07-01 11:46:10,"['odoo', 'sponsor', 'vidéo', 'découvrir', 'odo...",odoo sponsor vidéo découvrir odoo clique https...,Positive
1,UCK3inMNRNAVUleEbpDU1k2g,24Ob5JxhF4s,@guishoo4700,2025-06-30 18:45:12+00:00,Fait une vidéo sur Tyler the creator stp,2025-07-01 11:46:10,"['vidéo', 'tyler', 'creator']",vidéo tyler creator,Neutral
2,UCK3inMNRNAVUleEbpDU1k2g,24Ob5JxhF4s,@Remi-B-Goode,2025-06-30 16:35:49+00:00,Lol le disclaimer😂,2025-07-01 11:46:10,['disclaimer'],disclaimer,Neutral
3,UCK3inMNRNAVUleEbpDU1k2g,24Ob5JxhF4s,@Guaph,2025-06-29 20:17:58+00:00,"où peut-on voir la série ""Earn and Alfred"" ???",2025-07-01 11:46:10,"['voir', 'série', 'earn', 'alfred']",voir série earn alfred,Neutral
4,UCK3inMNRNAVUleEbpDU1k2g,24Ob5JxhF4s,@rockman156,2025-06-29 19:05:12+00:00,C'est pas un re upload ? Impression de déjà vu...,2025-07-01 11:46:10,"['upload', 'impression', 'critique', 'foireux'...",upload impression critique foireux impression,Positive


# sklearn

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [67]:
cv = CountVectorizer()
dtm = cv.fit_transform(df['tokens_clean_lem'])

LDA = LatentDirichletAllocation(n_components=5,random_state=42)
LDA.fit(dtm)

for index,topic in enumerate(LDA.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([cv.get_feature_names_out()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['childish', 'frank', 'community', 'ocean', 'découvrir', 'glover', 'faire', 'musique', 'donald', 'cheveux', 'jamais', 'vrai', 'coupe', 'bien', 'vidéo']


THE TOP 15 WORDS FOR TOPIC #1
['beaucoup', 'pouce', 'kendrick', 'petit', 'voir', 'aimer', 'incroyable', 'video', 'artiste', 'bien', 'trop', 'faire', 'childish', 'gambino', 'vidéo']


THE TOP 15 WORDS FOR TOPIC #2
['petit', 'disclaimer', 'ressembler', 'vraiment', 'fois', 'flow', 'album', 'artiste', 'clip', 'moche', 'dire', 'vidéo', 'coupe', 'arrière', 'cheveux']


THE TOP 15 WORDS FOR TOPIC #3
['community', 'super', 'donald', 'bien', 'regarder', 'faire', 'smith', 'glover', 'childish', 'jour', 'voir', 'connaître', 'série', 'artiste', 'vidéo']


THE TOP 15 WORDS FOR TOPIC #4
['childish', 'beaucoup', 'savoir', 'histoire', 'aimer', 'bien', 'video', 'faire', 'incroyable', 'vraiment', 'super', 'tyler', 'creator', 'artiste', 'vidéo']




# amélioration

# gensim

In [79]:
def corpus(df):
    # corpus = [c for c in df.tolist() if len(c)!=0]
    corpus = [eval(df.iloc[i]) for i in df.index if len(df.iloc[i]) > 0 ]
    print(f'Les 10 premiers documents du corpus sur les {len(corpus)} :\n {corpus[:5]}')
    unique_tokens = set(token for doc in corpus for token in doc)
    print(f'Nombre de tokens uniques dans le corpus: {len(unique_tokens)}, petit extrait des 10 premiers {list(unique_tokens)[:5]}')
    return corpus

In [78]:
corpus = corpus(df['tokens_clean_lem'])
corpus

Les 10 premiers documents du corpus sur les 1731 :
 [['odoo', 'sponsor', 'vidéo', 'découvrir', 'odoo', 'clique', 'httpsw.odoo.comrcal', 'découverte', 'redécouvrir', 'aller', 'écouter', 'childish', 'important', 'bise'], ['vidéo', 'tyler', 'creator'], ['disclaimer'], ['voir', 'série', 'earn', 'alfred'], ['upload', 'impression', 'critique', 'foireux', 'impression']]
Nombre de tokens uniques dans le corpus: 3274, petit extrait des 10 premiers ['salon', 'abusif', 'publicitaire', 'perspective', 'prouvable']


[['odoo',
  'sponsor',
  'vidéo',
  'découvrir',
  'odoo',
  'clique',
  'httpsw.odoo.comrcal',
  'découverte',
  'redécouvrir',
  'aller',
  'écouter',
  'childish',
  'important',
  'bise'],
 ['vidéo', 'tyler', 'creator'],
 ['disclaimer'],
 ['voir', 'série', 'earn', 'alfred'],
 ['upload', 'impression', 'critique', 'foireux', 'impression'],
 ['vidéo', 'tyler', 'creator', 'stp'],
 ['robot'],
 ['super',
  'vidéo',
  'trouver',
  'trop',
  'individualiste',
  'arriver',
  'faire',
  'grand',
  'partie',
  'fois',
  'entourer',
  'personne',
  'spécialiser',
  'domaine'],
 ['coiffure',
  'rasage',
  'déranger',
  'monde',
  'contre',
  'grésillement',
  'sortie',
  'fois',
  'immersion'],
 ['super',
  'vidéo',
  'vraiment',
  'contre',
  'faillir',
  'taper',
  'tête',
  'contre',
  'fois',
  'prononcer',
  'childish'],
 ['bonjour',
  'esper',
  'aller',
  'bien',
  'prier',
  'lire',
  'message',
  'venir',
  'savoir',
  'entrer',
  'contacter',
  'artiste',
  'côte',
  'ivoire',
  'didi

In [ ]:
# le dictionnaire
corpus = corpus(df['tokens_clean_lem'])
dictionary = corpora.Dictionary(corpus)
# Bag-of-words representation of the documents.
corpus_vect = [dictionary.doc2bow(doc) for doc in corpus]

print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus_vect))

# Set training parameters.
num_topics = 5
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make an index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus_vect,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)


Les 10 premiers documents du corpus sur les 1731 :
 [['odoo', 'sponsor', 'vidéo', 'découvrir', 'odoo', 'clique', 'httpsw.odoo.comrcal', 'découverte', 'redécouvrir', 'aller', 'écouter', 'childish', 'important', 'bise'], ['vidéo', 'tyler', 'creator'], ['disclaimer'], ['voir', 'série', 'earn', 'alfred'], ['upload', 'impression', 'critique', 'foireux', 'impression']]
Nombre de tokens uniques dans le corpus: 3274, petit extrait des 10 premiers ['salon', 'abusif', 'publicitaire', 'perspective', 'prouvable']
Number of unique tokens: 3274
Number of documents: 1731


In [81]:

for (topic, words) in model.print_topics():
    print("***********")
    print("* topic", topic+1, "*")
    print("***********")
    print(topic+1, ":", words)
    print()

***********
* topic 1 *
***********
1 : 0.029*"vidéo" + 0.028*"artiste" + 0.012*"bien" + 0.009*"trop" + 0.009*"dire" + 0.009*"faire" + 0.008*"connaître" + 0.006*"film" + 0.006*"histoire" + 0.006*"voir"

***********
* topic 2 *
***********
2 : 0.046*"vidéo" + 0.017*"tyler" + 0.017*"creator" + 0.013*"artiste" + 0.012*"vraiment" + 0.010*"faire" + 0.009*"super" + 0.008*"bien" + 0.007*"video" + 0.007*"trop"

***********
* topic 3 *
***********
3 : 0.044*"vidéo" + 0.014*"faire" + 0.010*"bien" + 0.010*"incroyable" + 0.008*"série" + 0.007*"glover" + 0.007*"childish" + 0.007*"vraiment" + 0.007*"donald" + 0.007*"atlanta"

***********
* topic 4 *
***********
4 : 0.028*"vidéo" + 0.024*"coupe" + 0.018*"cheveux" + 0.017*"childish" + 0.014*"gambino" + 0.010*"artiste" + 0.010*"arrière" + 0.008*"bien" + 0.008*"super" + 0.008*"faire"

***********
* topic 5 *
***********
5 : 0.022*"vidéo" + 0.012*"chill" + 0.011*"artiste" + 0.009*"dish" + 0.008*"kendrick" + 0.008*"flow" + 0.007*"troy" + 0.007*"lamar" + 0

In [82]:
import pyLDAvis.gensim

# Prepare the visualization
LDAvis_prepared = pyLDAvis.gensim.prepare(model, corpus_vect, dictionary)

# Display the visualization (if running in a Jupyter Notebook)
pyLDAvis.display(LDAvis_prepared)
# lambda metrci de pertinance, proche de 1 fréquence élevé 0 exclusivitié
# rouge fréquence dans topic 
# bleu fréquence dans le corpus 

In [90]:
def format_topics_sentences(ldamodel, corpus, texts):
    # Liste pour stocker les résultats
    topics_df = []

    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Topic dominant, % de contribution et mots-clés
        topic_num, prop_topic = row[0]
        topic_keywords = ", ".join([word for word, _ in ldamodel.show_topic(topic_num)])
        topics_df.append((i, int(topic_num), round(prop_topic, 4), topic_keywords, texts[i]))

    # Créer un DataFrame 
    df_topic_sents_keywords = pd.DataFrame(topics_df, columns=['Doc_ID', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Topic_Keywords', 'Text'])
    return df_topic_sents_keywords

In [92]:
res_lda = format_topics_sentences(ldamodel=model, corpus=corpus_vect, texts=df['tokens_clean_lem'])
res_lda.head()

,Doc_ID,Dominant_Topic,Topic_Perc_Contrib,Topic_Keywords,Text
0,0,2,0.9803,"vidéo, faire, bien, incroyable, série, glover,...","['odoo', 'sponsor', 'vidéo', 'découvrir', 'odo..."
1,1,1,0.9147,"vidéo, tyler, creator, artiste, vraiment, fair...","['vidéo', 'tyler', 'creator']"
2,2,2,0.7906,"vidéo, faire, bien, incroyable, série, glover,...",['disclaimer']
3,3,3,0.9391,"vidéo, coupe, cheveux, childish, gambino, arti...","['voir', 'série', 'earn', 'alfred']"
4,4,3,0.9505,"vidéo, coupe, cheveux, childish, gambino, arti...","['upload', 'impression', 'critique', 'foireux'..."


In [96]:
res_lda['Topic_Keywords'].value_counts()

Topic_Keywords
vidéo, coupe, cheveux, childish, gambino, artiste, arrière, bien, super, faire        507
vidéo, faire, bien, incroyable, série, glover, childish, vraiment, donald, atlanta    326
vidéo, tyler, creator, artiste, vraiment, faire, super, bien, video, trop             319
vidéo, artiste, bien, trop, dire, faire, connaître, film, histoire, voir              303
vidéo, chill, artiste, dish, kendrick, flow, troy, lamar, video, adorer               276
Name: count, dtype: int64

In [138]:
res_lda['Topic_Keywords'].unique().s

array(['vidéo, faire, bien, incroyable, série, glover, childish, vraiment, donald, atlanta',
       'vidéo, tyler, creator, artiste, vraiment, faire, super, bien, video, trop',
       'vidéo, coupe, cheveux, childish, gambino, artiste, arrière, bien, super, faire',
       'vidéo, chill, artiste, dish, kendrick, flow, troy, lamar, video, adorer',
       'vidéo, artiste, bien, trop, dire, faire, connaître, film, histoire, voir'],
      dtype=object)

In [143]:
# Evaluate LDA model
coherence = gensim.models.CoherenceModel(model=model, texts=corpus, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence.get_coherence()
print(f"Coherence score: {coherence_lda}")

Coherence score: 0.4523787810772465


# recherhce du nombre de topic optimal

In [170]:
def compute_coherence_values(dictionary, corpus, start, limit, step, texts):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit + 1, step):
        model = LdaModel(
                        corpus=corpus_vect,
                        id2word=id2word,
                        num_topics=num_topics,
                        eval_every=eval_every,
                        random_state=42
                    )
        # model = LdaModel(corpus=corpus,
        #                 id2word=dictionary,
        #                 alpha='auto',
        #                 eta='auto',
        #                 iterations=400,
        #                 num_topics=num_topics,
        #                 chunksize=2000,
        #                 passes=20,
        #                 eval_every=None
        #                 )
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, corpus=corpus, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
    return model_list, coherence_values




In [172]:
# Paramètres
start = 3
limit = 20
step = 1

# Calcul des valeurs de cohérence
model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=corpus_vect, texts=corpus, start=start, limit=limit, step=step)

# Affichage des résultats
x = range(start, limit + 1, step)

# Affichage des scores de cohérence
for m, cv in zip(x, coherence_values):
    print(f"Nombre de topics = {m}, Score de cohérence = {round(cv, 4)}")

# Trouver le nombre optimal de topics
optimal_model_index = np.argmax(coherence_values)
optimal_num_topics = x[optimal_model_index]
print(f"Nombre optimal de topics : {optimal_num_topics} avec un score de cohérence de {coherence_values[optimal_model_index]}")
# Affichage du modèle optimal
optimal_model = model_list[optimal_model_index]

Nombre de topics = 3, Score de cohérence = 0.4082
Nombre de topics = 4, Score de cohérence = 0.3857
Nombre de topics = 5, Score de cohérence = 0.4136
Nombre de topics = 6, Score de cohérence = 0.404
Nombre de topics = 7, Score de cohérence = 0.381
Nombre de topics = 8, Score de cohérence = 0.3992
Nombre de topics = 9, Score de cohérence = 0.4278
Nombre de topics = 10, Score de cohérence = 0.412
Nombre de topics = 11, Score de cohérence = 0.3958
Nombre de topics = 12, Score de cohérence = 0.393
Nombre de topics = 13, Score de cohérence = 0.411
Nombre de topics = 14, Score de cohérence = 0.4156
Nombre de topics = 15, Score de cohérence = 0.4032
Nombre de topics = 16, Score de cohérence = 0.4033
Nombre de topics = 17, Score de cohérence = 0.3856
Nombre de topics = 18, Score de cohérence = 0.3983
Nombre de topics = 19, Score de cohérence = 0.4083
Nombre de topics = 20, Score de cohérence = 0.3964
Nombre optimal de topics : 9 avec un score de cohérence de 0.4277932203036622


In [139]:
# from huggingface_hub import InferenceClient

# HF_token = open("../token_hf.txt", "r").read().strip()
# client = InferenceClient(
#     provider="novita",
#     api_key=HF_token,
# )

# completion = client.chat.completions.create(
#     model="mistralai/Mistral-7B-Instruct-v0.3",
#     messages=[
#         {"role": "system", 
#         "content": "Tu es un data scientist spécialisé en NLP."},
#         {
#             "role": "user ",
#             "content": " Résume en un mots les mots clés suivants : " + res_lda['Topic_Keywords'].unique()
#         }
#     ],temperature=0.0
# )

# print(completion.choices[0].message)

TypeError: Object of type ndarray is not JSON serializable

In [142]:
# for i in res_lda['Topic_Keywords'].value_counts().index.tolist():
#     print("________________Topic Keywords:", i)
#     completion = client.chat.completions.create(
#     model="mistralai/Mistral-7B-Instruct-v0.3",
#     messages=[
#         {"role": "system", 
#         "content": "Tu es un data scientist spécialisé en NLP."},
#         {
#             "role": "user ",
#             "content": " Nomme le sujet identifié par le modèle en te basant sur les mots suivants: " + i
#         }
#     ], temperature=0.0
# )
#     print(completion.choices[0].message.content)

________________Topic Keywords: vidéo, coupe, cheveux, childish, gambino, artiste, arrière, bien, super, faire
The subject identified by the model is "Childish Gambino, Artist, Rear View, Good, Super, Haircut, Video"

________________Topic Keywords: vidéo, faire, bien, incroyable, série, glover, childish, vraiment, donald, atlanta
 The subject identified by the model is a childish series called "Atlanta" featuring Donald Glover.

________________Topic Keywords: vidéo, tyler, creator, artiste, vraiment, faire, super, bien, video, trop
The subject identified by the model based on the given words is "creator" or "artiste".

________________Topic Keywords: vidéo, artiste, bien, trop, dire, faire, connaître, film, histoire, voir
 The subject identified by the model based on the given words could be a movie, artist, piece of art, film, story, or history that is too difficult to understand or know about without seeing it.

________________Topic Keywords: vidéo, chill, artiste, dish, kendrick,